In [1]:
import pandas as pd
import json

In [3]:
from shapely.geometry import Point
import libpysal
import esda

In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import numpy as np

import libpysal
from libpysal.weights import DistanceBand, KNN
from esda.getisord import G_Local
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# JSON 파일 불러오기
with open("violations.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# DataFrame으로 변환
df = pd.DataFrame(data)

# 데이터 확인
print(df.head())

         lat         lon  hour type  count
0  37.259503  127.060575     8  주행형   2031
1  37.264954  127.029911     7  고정형   1274
2  37.264954  127.029911     9  고정형   1052
3  37.262671  127.035840    14  고정형    991
4  37.259503  127.060575    10  주행형    891


In [10]:
CRS_WGS84 = "EPSG:4326"
CRS_KOREA_M = "EPSG:5186"   # 미터 단위 (한국 TM)
DIST_THRESHOLD_M = 2000      # 이웃 반경 (m) — 필요시 700~1000으로 조정
ALPHA = 0.05
Z_CUTOFF = 1.96
MIN_POINTS = 5              # 시간대별 최소 포인트 (너무 적으면 스킵)

# 안전 체크
assert {"lat","lon","hour","count"}.issubset(df.columns), "필요 컬럼이 없습니다."

def compute_gistar_drop_islands(df_hour: pd.DataFrame) -> gpd.GeoDataFrame | None:
    """고립 포인트 제외 후, 해당 시간대의 Gi* 계산 결과를 반환 (WGS84 좌표계)"""
    # Geo 변환
    gdf = gpd.GeoDataFrame(
        df_hour.copy(),
        geometry=gpd.points_from_xy(df_hour["lon"], df_hour["lat"]),
        crs=CRS_WGS84
    )
    gdf_m = gdf.to_crs(CRS_KOREA_M)

    # 초기 가중치(반경 기반)
    coords = np.array([[pt.x, pt.y] for pt in gdf_m.geometry])
    w = DistanceBand(coords, threshold=DIST_THRESHOLD_M, binary=True, silence_warnings=True)

    # 고립 포인트(이웃 0개) 제거
    islands = [i for i, nbs in w.neighbors.items() if len(nbs) == 0]
    if len(islands) > 0:
        keep = np.ones(len(gdf_m), dtype=bool)
        keep[islands] = False
        gdf_m = gdf_m.loc[keep].copy()
        gdf = gdf.loc[keep].copy()

    if len(gdf_m) < max(MIN_POINTS, 4):
        return None

    # 재가중치(제외 후 다시 거리 기반)
    coords2 = np.array([[pt.x, pt.y] for pt in gdf_m.geometry])
    w2 = DistanceBand(coords2, threshold=DIST_THRESHOLD_M, binary=True, silence_warnings=True)

    # Gi* 계산
    y = gdf_m["count"].astype(float).values
    gi = G_Local(y, w2)

    gdf["GiZ"] = gi.Zs
    gdf["GiP"] = gi.p_sim
    gdf["label"] = np.where((gdf["GiP"] < ALPHA) & (gdf["GiZ"] > Z_CUTOFF), "Hotspot",
                     np.where((gdf["GiP"] < ALPHA) & (gdf["GiZ"] < -Z_CUTOFF), "Coldspot", "Not Significant"))
    return gdf  # WGS84


In [11]:
available_hours = []
for h in sorted(df["hour"].dropna().unique().tolist()):
    sub = df[df["hour"] == h].copy()
    res = compute_gistar_drop_islands(sub)
    if res is None:
        print(f"[hour={h}] 유효 포인트 부족으로 스킵")
        continue

    res["hour"] = h
    # 카카오맵에서 쓰기 쉬운 필드만 저장
    out = res[["lat","lon","hour","count","GiZ","GiP","label"]].copy()
    out.to_json(f"hotspots_hour_{h}.json", orient="records", force_ascii=False)
    available_hours.append(int(h))

# 어떤 시간대 파일이 만들어졌는지 매니페스트 저장 (카카오맵에서 슬라이더/버튼 구성 용)
pd.Series(sorted(available_hours)).to_json("available_hours.json", orient="values")
print("완료:", sorted(available_hours))

완료: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]


In [24]:
# -*- coding: utf-8 -*-
# pip install geopandas libpysal esda shapely pyproj

import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from libpysal.weights import KNN
from esda.getisord import G_Local

# =========================================================
# 입력 df가 이미 메모리에 있다고 가정: lat, lon, hour, count
# df = pd.read_json("your_input.json")
# =========================================================

# -----------------------------
# 파라미터
# -----------------------------
CRS_WGS84 = "EPSG:4326"
CRS_KM    = "EPSG:5186"
PERM      = 999
MIN_POINTS = 5

# KNN 이웃 수(전역 고정)
K_NEI = 8          # 6~12 권장. 작을수록 지역성↑, 클수록 매끈해짐

# 필터 기준(완화)
P_THRESH        = 0.15
Z_THRESH        = 1.00
LOCAL_SUM_Q     = 0.35
CENTER_COUNT_Q  = 0.35
MIN_NEI         = 1          # KNN이라 항상 >=1이므로 큰 의미는 없음 (안전용)
MIN_SEP_M       = 0          # thinning off

# 폴백(최소 개수 보장)
FALLBACK_MIN_N  = 30
FALLBACK_P      = 0.25
FALLBACK_Z      = 0.67
FALLBACK_LS_Q   = 0.30
FALLBACK_CC_Q   = 0.30

# 구간 정의
PERIODS = {
    "morning":   {"label":"오전",  "hours": list(range(6,11))},
    "lunch":     {"label":"점심",  "hours": list(range(11,15))},
    "afternoon": {"label":"오후",  "hours": list(range(15,18))},
    "evening":   {"label":"저녁",  "hours": list(range(18,23))},
}

# 반지름(시각화)
def radius_from(local_sum, z):
    z_term = max(0.0, z)
    r = 6 + 0.45*np.sqrt(max(local_sum, 0)) + 1.8*z_term
    return float(min(40, max(6, r)))

# -----------------------------
# 전역 지점 집합 + 전역 KNN W 구성
# -----------------------------
assert {"lat","lon","hour","count"}.issubset(df.columns), "필요 컬럼 누락"

# 1) 전 시간대 통틀어 "고유 좌표" 집합 확보 (지점 집합 고정)
sites = df[["lat","lon"]].drop_duplicates().reset_index(drop=True)
if len(sites) < MIN_POINTS:
    raise SystemExit("유효 지점이 너무 적습니다.")

g_sites = gpd.GeoDataFrame(
    sites.copy(),
    geometry=gpd.points_from_xy(sites.lon, sites.lat),
    crs=CRS_WGS84
).to_crs(CRS_KM)

coords = np.column_stack([g_sites.geometry.x.values, g_sites.geometry.y.values])

# 2) 전역 KNN W (행표준화)
W = KNN.from_array(coords, k=K_NEI)
W.transform = "R"

# 3) 좌표 → 행 인덱스 매핑
site_index = { (row.lat, row.lon): i for i, row in sites.iterrows() }

# -----------------------------
# 공통 Gi* + 로컬합 계산 함수 (전역 W 사용)
# -----------------------------
def gi_and_local_sum_with_global_W(df_xyc: pd.DataFrame):
    """
    입력: df_xyc(해당 구간의 좌표별 count 합계)
    처리: 전역 sites 순서에 맞춰 길이 N 벡터 y를 만들고 Gi* 계산
    반환: (WGS84 GeoDataFrame with GiZ, GiP, local_sum, local_n, center_count)
    """
    # y 벡터 준비(모든 sites에 대해 0으로 시작)
    N = len(sites)
    y = np.zeros(N, dtype=float)

    if not df_xyc.empty:
        # 구간 내 합산된 count를 전역 인덱스에 채워넣기
        # (좌표가 전역 집합에 반드시 포함된다고 가정; 혹시 소수점 오차가 있다면 반경-조인 필요)
        for _, r in df_xyc.iterrows():
            idx = site_index.get((r["lat"], r["lon"]))
            if idx is not None:
                y[idx] = float(r["count"])

    # Gi*
    gi = G_Local(y, W, permutations=PERM)

    # 로컬합(자기+이웃)
    neighbors = W.neighbors
    local_sum = []
    local_n   = []
    for i in range(N):
        nbs = neighbors.get(i, [])
        idxs = [i] + nbs
        s = float(np.sum(y[idxs]))
        local_sum.append(s)
        local_n.append(len(nbs))

    out = g_sites.to_crs(CRS_WGS84).copy()
    out["GiZ"], out["GiP"]      = gi.Zs, gi.p_sim
    out["local_sum"], out["local_n"] = local_sum, local_n
    out["center_count"]         = y
    return out

# -----------------------------
# 필터(진단 로그 포함, .loc 사용)
# -----------------------------
def _apply_filters(gdf, p_th, z_th, ls_q, cc_q, min_nei, min_sep_m):
    g0 = gdf.copy()
    print(f"[필터] p<{p_th}, Z>{z_th}, lsQ≥{ls_q}, ccQ≥{cc_q}, min_nei≥{min_nei}, sep={min_sep_m}m")
    print("  입력:", len(g0))

    g1 = g0[(g0["GiP"] < p_th) & (g0["GiZ"] > z_th) & (g0["local_n"] >= min_nei)].copy()
    print("  통계/이웃 통과:", len(g1))
    if g1.empty: return g1

    base = g0[g0["local_n"] >= 1].copy() if "local_n" in g0.columns else g0
    ls_cut = float(np.nanquantile(base["local_sum"], ls_q)) if len(base) > 3 else base["local_sum"].median()
    cc_cut = float(np.nanquantile(base["center_count"], cc_q)) if len(base) > 3 else base["center_count"].median()

    g2 = g1[(g1["local_sum"] >= ls_cut) & (g1["center_count"] >= cc_cut)].copy()
    print(f"  규모컷 통과 (base: local_sum≥Q{ls_q:.2f}={ls_cut:.1f}, center≥Q{cc_q:.2f}={cc_cut:.1f}):", len(g2))
    if g2.empty: return g2

    if min_nei and min_sep_m > 0:
        gm = g2.to_crs(CRS_KM)
        coords2 = np.column_stack([gm.geometry.x.values, gm.geometry.y.values])
        order = np.argsort(-g2["GiZ"].values)
        keep, used = [], np.zeros(len(g2), dtype=bool)
        for i in order:
            if used[i]: continue
            keep.append(i)
            d = np.hypot(coords2[:,0]-coords2[i,0], coords2[:,1]-coords2[i,1])
            used |= (d < min_sep_m)
        g2 = g2.iloc[keep].copy()
        print(f"  thinning 후({min_sep_m}m):", len(g2))

    g2.loc[:, "color"]  = "#e53935"
    g2.loc[:, "radius"] = [radius_from(s, z) for s, z in zip(g2["local_sum"], g2["GiZ"])]
    return g2

def filter_hot(out_gdf: gpd.GeoDataFrame) -> gpd.GeoDataFrame:
    g = _apply_filters(out_gdf, P_THRESH, Z_THRESH, LOCAL_SUM_Q, CENTER_COUNT_Q, MIN_NEI, MIN_SEP_M)

    if g is None or len(g) < FALLBACK_MIN_N:
        print(f"⚠️ 폴백 적용: 목표 {FALLBACK_MIN_N}개 확보 시도")
        g_fb = _apply_filters(out_gdf, FALLBACK_P, FALLBACK_Z, FALLBACK_LS_Q, FALLBACK_CC_Q, 1, 0)
        if g_fb is not None and len(g_fb) > (0 if g is None else len(g)):
            g = g_fb

    if g is None or len(g) < FALLBACK_MIN_N:
        x = out_gdf.copy()
        x.loc[:, "z_norm"]  = (x["GiZ"] - x["GiZ"].min()) / (x["GiZ"].max() - x["GiZ"].min() + 1e-9)
        x.loc[:, "ls_norm"] = (x["local_sum"] - x["local_sum"].min()) / (x["local_sum"].max() - x["local_sum"].min() + 1e-9)
        x.loc[:, "cc_norm"] = (x["center_count"] - x["center_count"].min()) / (x["center_count"].max() - x["center_count"].min() + 1e-9)
        x.loc[:, "score"]   = 0.5*x["z_norm"] + 0.3*x["ls_norm"] + 0.2*x["cc_norm"]
        x = x.sort_values("score", ascending=False).head(FALLBACK_MIN_N).copy()
        x.loc[:, "color"]  = "#e53935"
        x.loc[:, "radius"] = [radius_from(s, z) for s, z in zip(x["local_sum"], x["GiZ"])]
        print(f"  Top-N 보장 적용 → {len(x)}개")
        g = x
    return g

# -----------------------------
# 메인: 전역 W로 고정한 상태에서 구간별 y만 교체
# -----------------------------
period_meta = []

for key, meta in PERIODS.items():
    hrs = meta["hours"]
    sub = df[df["hour"].isin(hrs)].copy()
    if sub.empty:
        print(f"[{meta['label']}] 데이터 없음 → 스킵")
        continue

    # 구간 내 동일 좌표 count 합산 (좌표는 전역 sites 중 일부일 수 있음)
    sub_agg = sub.groupby(["lat","lon"], as_index=False)["count"].sum()

    # Gi* + 로컬합 (전역 W 사용)
    out = gi_and_local_sum_with_global_W(sub_agg)

    # Hotspot 필터
    hot = filter_hot(out)
    if hot is None or hot.empty:
        print(f"[{meta['label']}] Hotspot 없음 → 스킵")
        continue

    hot = hot.copy()
    hot.loc[:, "period_key"]   = key
    hot.loc[:, "period_label"] = meta["label"]

    export = hot[["lat","lon","period_key","period_label",
                  "center_count","local_sum","local_n","GiZ","GiP","color","radius"]].copy()
    export.rename(columns={"center_count":"count"}, inplace=True)
    export.to_json(f"hotspots_period_{key}_hot.json", orient="records", force_ascii=False)

    period_meta.append({"key": key, "label": meta["label"], "n": int(len(export))})
    print(f"[{meta['label']}] 저장: hotspots_period_{key}_hot.json (전역 KNN k={K_NEI}, 최종 {len(export)})")

pd.DataFrame(period_meta).to_json("available_periods_hot.json", orient="records", force_ascii=False)
print("완료: available_periods_hot.json 및 각 구간 hot JSON 생성")


c:\Users\wonny\anaconda3\Lib\site-packages\libpysal\weights\distance.py:153: UserWarning: The weights matrix is not fully connected: 
 There are 2 disconnected components.
  W.__init__(self, neighbors, id_order=ids, **kwargs)


[필터] p<0.15, Z>1.0, lsQ≥0.35, ccQ≥0.35, min_nei≥1, sep=0m
  입력: 14871
  통계/이웃 통과: 179
  규모컷 통과 (base: local_sum≥Q0.35=7.0, center≥Q0.35=0.0): 179
[오전] 저장: hotspots_period_morning_hot.json (전역 KNN k=8, 최종 179)
[필터] p<0.15, Z>1.0, lsQ≥0.35, ccQ≥0.35, min_nei≥1, sep=0m
  입력: 14871
  통계/이웃 통과: 320
  규모컷 통과 (base: local_sum≥Q0.35=8.0, center≥Q0.35=0.0): 320
[점심] 저장: hotspots_period_lunch_hot.json (전역 KNN k=8, 최종 320)
[필터] p<0.15, Z>1.0, lsQ≥0.35, ccQ≥0.35, min_nei≥1, sep=0m
  입력: 14871
  통계/이웃 통과: 189
  규모컷 통과 (base: local_sum≥Q0.35=6.0, center≥Q0.35=0.0): 189
[오후] 저장: hotspots_period_afternoon_hot.json (전역 KNN k=8, 최종 189)
[필터] p<0.15, Z>1.0, lsQ≥0.35, ccQ≥0.35, min_nei≥1, sep=0m
  입력: 14871
  통계/이웃 통과: 154
  규모컷 통과 (base: local_sum≥Q0.35=6.0, center≥Q0.35=0.0): 154
[저녁] 저장: hotspots_period_evening_hot.json (전역 KNN k=8, 최종 154)
완료: available_periods_hot.json 및 각 구간 hot JSON 생성
